 **Configure environment**

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

**Node configuration**

In [ ]:
slice_name="scherrer-original-" + fablib.get_bastion_username()

node_conf = [
 {'name': "mininet_node",   'cores': 4, 'ram': 16, 'disk': 100, 'image': 'default_ubuntu_22', 'packages': []}
]

exp_conf = {'cores': sum([ n['cores'] for n in node_conf]) }

**Reserve resources**

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) ):
        break

fablib.show_site(site_name)

Then we will add the host

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

* While it is being prepared, the “State” of the slice will appear as “Configuring”.
* When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()


In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

**Log into resources**

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

**Install Mininet**

In [ ]:
slice.get_node(name="mininet_node").execute('git clone https://github.com/mininet/mininet')
slice.get_node(name="mininet_node").execute('mininet/util/install.sh -a')
slice.get_node(name="mininet_node").execute('sudo mn --switch ovsbr --test pingall')

**Clone original repo and install dependencies**

In [ ]:
slice.get_node(name="mininet_node").execute('git clone https://github.com/ANONYMIZED/imc22-bbr-fluid-model.git')
slice.get_node(name="mininet_node").execute('cd imc22-bbr-fluid-model; sudo bash ./install_dependencies.sh')
slice.get_node(name="mininet_node").execute('sudo apt-get -y install texlive-latex-extra')

**Run BBR model checking experiment**

In [ ]:
slice.get_node(name="mininet_node").execute('cd imc22-bbr-fluid-model;./run_model.py -c configs/bbr_model_checking.yaml')

In [ ]:
cmds_run_bbr_model_checking = '''
            cd imc22-bbr-fluid-model
            ./run_model.py -c configs/bbr_model_checking.yaml
            sudo ./run_experiments.py configs/bbr_model_checking.yaml
            ./plot_trace.py configs/bbr_model_checking.yaml
            ./plot.py configs/bbr_model_checking.yaml
            '''

slice.get_node(name="mininet_node").execute(cmds_run_bbr_model_checking)

In [ ]:
slice.get_node(name="mininet_node").download_file("/home/fabric/work/trace_original.png","/imc22-bbr-fluid-model/results/bbr_model_checking/mininet_experiments/results/12/1/100/0.5/BBR-1/2024-03-31--19-56-38/throughput_dist.png")





**Run 100 Mbps, 10 ms experiments**

In [ ]:
cmds_run_scenario = '''
            cd imc22-bbr-fluid-model
            ./run_model.py -c configs/model_validation_100Mbps_5ms.yaml
            sudo ./run_experiments.py configs/model_validation_100Mbps_5ms.yaml
            ./plot_trace.py configs/model_validation_100Mbps_5ms.yaml
            ./plot.py configs/model_validation_100Mbps_5ms.yaml
            '''

slice.get_node(name="mininet_node").execute(cmds_run_scenario)